# Data Preparation and cleaning

## Libraries

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

import os
import matplotlib.pyplot as plt

%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [ ]:
orders_path = '../orders_cleaned.csv'

df_orders = pd.read_csv(orders_path,
                        dtype=
                        {'Customer number': object,
                         'Gender': 'category',
                         'PoR': object,
                         'Order number': object,
                         'Product number': object,
                         'Sub category': 'category',
                         'Category': 'category',
                         'Count': np.int16,
                         'Price': np.float64,
                         'EDT': np.int16,
                         'ADT': np.int16,
                         'RoR': object,
                         'Rating': np.int8})

df_orders["Order date"] = pd.to_datetime(df_orders["Order date"], format="%d-%m-%Y")
df_orders["DoB"] = pd.to_datetime(df_orders["DoB"], format="%d-%m-%Y")

## Feature engineering

### "Returned"
    -> binary value
    -> is the product retured or not

In [ ]:
df_orders.loc[df_orders['RoR'].isnull(), 'Returned'] = 0
df_orders.loc[df_orders['RoR'].notnull(), 'Returned'] = 1
df_orders = df_orders.drop('RoR', axis=1)

### "Price per product"
    -> what is the price of the product ordered

In [ ]:
df_orders['Price per Product'] = df_orders['Price'] / df_orders['Count']

### "Man" / "Woman"
    -> boolean values
    -> is the gender man or woman
    -> the method is called One-Hot-Encoding. It is done instead of changing the genders to 0 and 1 in the original column, so the models in the future can diferentiate them like categories, not numbers
    -> as there are statistically unimportant number of missing gender values, we will make both columns 'Man' and 'Woman' = 0

In [ ]:
df_orders.loc[df_orders['Gender'] == "Man", 'Man'] = 1
df_orders.loc[df_orders['Gender'] == "Woman", 'Man'] = 0
df_orders.loc[df_orders['Gender'] == "Man", 'Woman'] = 0
df_orders.loc[df_orders['Gender'] == "Woman", 'Woman'] = 1

In [ ]:
df_orders['Man'].replace(np.nan, 0, inplace=True)
df_orders['Woman'].replace(np.nan, 0, inplace=True)

### "Age" 
    -> based on the date of birth

In [ ]:
today = dt.datetime.today()

df_orders["DoB"] = pd.to_datetime(df_orders["DoB"], format="%d-%m-%Y")
df_orders["Age"] = today.year - df_orders["DoB"].dt.year - ((today.month <= df_orders["DoB"].dt.month)
                                                            & (today.day <= df_orders["DoB"].dt.day))

### "Order month" and "Order year"
    -> separate the month and the year of an order for easier analysis

In [ ]:
df_orders["Order date"] = pd.to_datetime(df_orders["Order date"], format="%d-%m-%Y")
df_orders["Order month"] = df_orders["Order date"].dt.month
df_orders["Order year"] = df_orders["Order date"].dt.year

### "DeltaT"
    -> number representation of the days
    -> the 0 day is 01-01-2013 and each next day is +1

In [ ]:
first_order = pd.to_datetime("01-01-2013", format="%d-%m-%Y")
df_orders["DeltaT"] = (df_orders["Order date"] - first_order).dt.days

## Current information about the dataset

In [ ]:
df_orders.info()

# Exploratory Data Analysis (EDA)

## Gender distribution

In [ ]:
unique_gender = df_orders[['Customer number', 'Man', 'Woman']]
unique_gender = unique_gender.drop_duplicates('Customer number')

male = unique_gender[unique_gender['Man'] == 1]
female = unique_gender[unique_gender['Woman'] == 1]

In [ ]:
labels = 'Male', 'Female'
sizes = [len(male), len(female)]

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
ax1.axis('equal')

plt.show()

## Age distribution

In [ ]:
customers_age = df_orders[['Customer number', 'Age']]
customers_age = customers_age.drop_duplicates('Customer number')

In [ ]:
sns.set_style('darkgrid')
x = pd.Series(customers_age['Age'], name="Ages")
plt.subplots(figsize=(10,6))
sns.distplot(x)
plt.title("Distribution of ages")
print()

## Categories and subcategories

In [ ]:
df_ordered_categories = df_orders.groupby(['Category', 'Sub category'])
print(df_ordered_categories[['Category', 'Sub category']].nunique().to_string())

## Money earned per category

In [ ]:
money_per_category = df_orders.groupby(['Category'])['Price'].sum()
pd.set_option('display.float_format', lambda x: '%.3f' % x)
money_per_category

## Distribution of orders per customer

In [ ]:
df_orders = df_orders.sort_values(['Customer number', 'Order date'])

orders_per_customer = df_orders[['Customer number', 'Order number']]
orders_per_customer = orders_per_customer.groupby('Customer number')['Order number'].count().reset_index()
orders_per_customer.rename(columns = {'Order number' : 'Number of orders'}, inplace=True)

In [ ]:
sns.set_style('darkgrid')
x = pd.Series(orders_per_customer['Number of orders'], name="Order count")
plt.subplots(figsize=(10,6))
sns.distplot(x)
plt.title("Distribution of orders")
print()

## Replacing all -1 values with NaN in 'Rating'

In [ ]:
df_orders['Rating'].replace(-1, np.nan, inplace=True)

## Average rating per customer

In [ ]:
rating_per_customer_avg = df_orders[['Customer number', 'Rating']]
rating_per_customer_avg.head()

In [ ]:
rating_per_customer_avg = rating_per_customer_avg.groupby('Customer number')['Rating'].mean().reset_index()

### Give all people that haven't given raiting an average raiting value of -1

In [ ]:
rating_per_customer_avg['Rating'].replace(np.nan, -1, inplace=True)
rating_per_customer_avg.rename(columns = {'Rating' : 'Average rating'}, inplace=True)

## Estimated date of arrival / actual date of arrival

In [ ]:
df_average_edt_divided_by_adt_per_customer = df_orders[['Customer number', 'EDT', 'ADT']]

In [ ]:
df_average_edt_divided_by_adt_per_customer = df_average_edt_divided_by_adt_per_customer.groupby('Customer number')['EDT', 'ADT'].mean().reset_index()

In [ ]:
df_average_edt_divided_by_adt_per_customer.head()

In [ ]:
df_average_edt_divided_by_adt_per_customer['EDT_divided_ADT'] = df_average_edt_divided_by_adt_per_customer['EDT'] / df_average_edt_divided_by_adt_per_customer['ADT']

In [ ]:
df_average_edt_divided_by_adt_per_customer.head()

## Percent returned items

In [ ]:
returns_per_customer = df_orders[['Customer number', 'Returned']]
total_returns = returns_per_customer.groupby('Customer number')['Returned'].count()
actual_returns = returns_per_customer[returns_per_customer['Returned'] == 1].groupby('Customer number')['Returned'].count()

returns_per_customer = returns_per_customer.drop_duplicates('Customer number')
returns_per_customer = returns_per_customer.drop('Returned', axis=1)

percentage_returned = actual_returns.divide(total_returns, fill_value=0).to_frame()
returns_per_customer = pd.merge(returns_per_customer, percentage_returned, how='left', on='Customer number')

## Churn analysis

### Get dates of orders per customer

In [ ]:
dates_per_customer = df_orders[['Customer number', 'Order number', 'Order date']]
dates_per_customer = dates_per_customer.drop_duplicates('Order number')
dates_per_customer = dates_per_customer.drop('Order number', axis=1)

In [ ]:
customer_dates = dates_per_customer.groupby('Customer number')['Order date'].apply(lambda x: x.tolist())
customer_dates_dict = customer_dates.to_dict()
customer_dates = customer_dates.reset_index()

### Calculate orders' date difference to determine churn
    -> the algorithm calculates the days difference between two consecutive orders and gets the mean of all differences + the standart deviation of all differences
    -> based on that we later determine if the customer has churned, using the date of his/hers last order

In [ ]:
def calculate_mean(dates):
    days = np.empty(len(dates) - 1)
    for i, date in enumerate(dates):
        if i < len(dates) - 1:
            days[i] = ((dates[i + 1] - dates[i]).days)

    return round(days.mean()+days.std())

In [ ]:
churn_array = np.empty(len(customer_dates_dict.items()))
meanstd_time_between_orders_array = np.empty(len(customer_dates_dict.items()))

for i, (key, value) in enumerate(customer_dates_dict.items()):
    meanstd_time_between_orders = calculate_mean(value)
    if (pd.Timestamp(2017, 11, 26) - customer_dates_dict[key][-1]).days > meanstd_time_between_orders:
        churn_array[i] = 1
    else:
        churn_array[i] = 0
    meanstd_time_between_orders_array[i] = meanstd_time_between_orders

## Drop unessecary featutures and get unique customer numbers

In [ ]:
df_orders = df_orders.drop(
    ['Order number', 'Order date', 'EDT', 'ADT', 'Rating', 'Returned'],
    axis=1).drop_duplicates().reset_index().drop('index', axis=1)

## Create a dateset for customer churn analysis

In [ ]:
customer_dates['T_between_orders'] = meanstd_time_between_orders_array
customer_dates['Number of orders'] = orders_per_customer['Number of orders']
customer_dates['Average rating'] = rating_per_customer['Average rating']
customer_dates['Average returns'] = returns_per_customer['Returned']
customer_dates['EDT'] = df_average_edt_divided_by_adt_per_customer['EDT']
customer_dates['ADT'] = df_average_edt_divided_by_adt_per_customer['ADT']
customer_dates['EDT_divided_ADT'] = df_average_edt_divided_by_adt_per_customer['EDT_divided_ADT']
customer_dates['Man'] = df_orders['Man']
customer_dates['Woman'] = df_orders['Woman']
customer_dates['Age'] = df_orders['Age']
customer_dates['Churn'] = churn_array
customer_dates.drop('Order date', axis=1, inplace=True)
customer_dates.dropna(inplace=True)
customer_dates.head()

## Plots

In [ ]:
labels = ['Churned', 'Not churned']
fig, ax = plt.subplots()
x = customer_dates[customer_dates['Churn'] == 1]['Churn'].count()
y = customer_dates[customer_dates['Churn'] == 0]['Churn'].count()
ax.pie([x, y], labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)
ax.axis('equal')

plt.show()

In [ ]:
# data to plot
n_groups = 1
mean_edt = customer_dates['EDT'].mean()
mean_adt = customer_dates['ADT'].mean()

# create plot
fig, ax = plt.subplots()
index = np.arange(n_groups)
bar_width = 0.35
opacity = 0.8
 
rects1 = plt.bar(index, mean_edt, 0.1,
                 alpha=opacity,
                 color='b',
                 label='EDT')
 
rects2 = plt.bar(index + 0.11, mean_adt, 0.1,
                 alpha=opacity,
                 color='g',
                 label='ADT')
 
plt.xlabel('Estimated T of delivery / Actual T of delivery')
plt.ylabel('Days')
plt.title('Comparison of average EDT and ADT')
plt.tick_params(bottom=False, labelbottom=False)
plt.legend()

plt.show()

In [ ]:
ax = sns.distplot(customer_dates['T_between_orders'], kde=False)
ax.set(xlabel='Average days between orders', ylabel='Customers')
plt.show()

In [ ]:
ax = sns.distplot(customer_dates['Percentage of returns'], kde=False)
ax.set(ylabel='Customers')
plt.show()

In [ ]:
ax = sns.distplot(customer_dates['Average rating'], kde=False, 
                  hist_kws={'range':[1.75,5.5]})
ax.set(ylabel='Customers')
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(10, 8))
corr = customer_dates.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax)